# answer

### import package

In [22]:
import os
import torch
import numpy as np
import math
import glob
from os import listdir
from os import walk
from torch import nn
from tqdm import tqdm 
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from torchvision import datasets, transforms
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization

import PIL.Image as Image
from matplotlib import pyplot as plt
import torch.nn.functional as F
import torchvision.models as models


### check gpu

In [14]:
data_path = '../../dataset/bird_datasets/test'
classes_path = '../../dataset/bird_datasets/classes.txt'
test_filename_path = '../../dataset/bird_datasets/testing_img_order.txt'

print('torch version:' + torch.__version__)

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Available GPUs: ', end='')
    for i in range(torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i), end=' ')
else:
    device = torch.device('cpu')
    print('CUDA is not available.')

torch version:1.8.0
Available GPUs: GeForce RTX 2080 Ti GeForce GTX 1080 Ti 

### Data augmentation

In [21]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

def get_trnsform():
    transform = transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        normalize
    ])
    return transform
trans = get_trnsform()

In [30]:
model_foloder = 'model/model_bird_resnet50'
with torch.no_grad():
    model = models.resnet50(pretrained=True)
    model.load_state_dict(torch.load('{}/checkpoint_0020.pth.tar'.format(model_foloder))['state_dict'])
    model.eval().to(device)

In [43]:
def predict(path):
    img = Image.open(os.path.join(data_path, path)).convert('RGB')
    m = nn.Softmax(dim=1)
    data = trans(img).unsqueeze(0).to(device)
    y = model(data)
    output = torch.argmax(m(y)) + 1
    return output 

In [ ]:
with open(test_filename_path) as f:
     test_images = [x.strip() for x in f.readlines()]  # all the testing images

submission = []
for img in tqdm(test_images):  # image order is important to your result
    predicted_class = predict(img)
    submission.append([img, predicted_class])

np.savetxt('answer.txt', submission, fmt='%s')

 75%|███████▍  | 2270/3033 [00:59<00:21, 35.66it/s]